In [1]:
import sys, os
import argparse

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../")))

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))

In [2]:
from fedml_api.data_preprocessing.cifar10.data_loader import load_partition_data_cifar10
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer as MyModelTrainerCLS
# from fedml_api.model.cv.resnet import resnet56
from fedml_api.model.contrastive_cv.resnet_with_embedding import Resnet56

import torch
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random
import pickle

In [3]:
dataset = 'cifar10'
data_dir = "./../../../data/cifar10"
# partition_method = 'hetero'
partition_method = 'homo'
partition_alpha = 0.5
client_num_in_total = 3
batch_size = 100
total_epochs = 500
save_model_path = 'model/client_{0}_triplet_epochs_{1}.pt'

device = 'cuda:2'

In [4]:
with open(f'dataset_{partition_method}_{client_num_in_total}.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [5]:
class Client(object):
    def __init__(self, client_index, train_data_local_dict, train_data_local_num_dict, test_data_local_dict, device, model):
        self.id = client_index
        self.train_data = train_data_local_dict[self.id]
        self.local_sample_number = train_data_local_num_dict[self.id]
        self.test_local = test_data_local_dict[self.id]
        
        self.device = device
        self.model = model

In [6]:
model = Resnet56(class_num=dataset[-1], neck='bnneck')
model.load_state_dict(torch.load(str.format('model/cs_{0}_{1}_client_{2}_oral_epochs_{3}.pt', client_num_in_total, partition_method, 0, 200)))
# [train_data_num, test_data_num, train_data_global, test_data_global, \
# #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# #             class_num]
client_1 = Client(0, dataset[5], dataset[4], dataset[6], device, model)
# global test data
# client_1.test_local = dataset[3]

In [7]:
def test(model, test_data, device):
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
            pred, feat = model(x)
            loss = criterion(pred, target)

            _, predicted = torch.max(pred, -1)
            correct = predicted.eq(target).sum()

            metrics['test_correct'] += correct.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
            
    return metrics

In [10]:
metrics = test(client_1.model, client_1.test_local, client_1.device)
test_correct = metrics['test_correct']/metrics['test_total']
test_loss = metrics['test_loss']/metrics['test_total']
print(metrics['test_total'])
print(f'test_correct: {test_correct}; test_loss: {test_loss}')

10000
test_correct: 0.9036; test_loss: 0.3458588407933712
